In [1]:
import os
import numpy
import re
import random
from scipy.stats import gaussian_kde, pearsonr
from sklearn.metrics import mean_squared_error, r2_score
import math
import matplotlib.pyplot as plt

In [2]:
WORK_SPACE = '/disk1/workspace/20230707/03_Ray_RAA'

In [4]:
MONTH_LABEL = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

## 筛选后的

In [11]:
    folder_type_list = ['RAA', 'Ray']
    folder_l1_list = ['26']
    folder_l2_list = ['0']

    for folder_type in folder_type_list:
        print(folder_type)
        folder_type_path = os.path.join(WORK_SPACE, folder_type)
        for folder_l1 in folder_l1_list:
            folder_l1_path = os.path.join(folder_type_path, folder_l1)
            month_slope_b3 = numpy.zeros((12,))
            month_r_b3 = numpy.zeros((12,))
            month_rmse_b3 = numpy.zeros((12,))
            month_offset_b3 = numpy.zeros((12,))
            month_slope_b4 = numpy.zeros((12,))
            month_r_b4 = numpy.zeros((12,))
            month_rmse_b4 = numpy.zeros((12,))
            month_offset_b4 = numpy.zeros((12,))
            count_record = []
            for month_idx in range(len(MONTH_LABEL)):
                month = MONTH_LABEL[month_idx]
                # each png
                misr_SR_band3_item_list = []
                ahi_SR_band3_item_list = []
                misr_SR_band4_item_list = []
                ahi_SR_band4_item_list = []
                for folder_l2 in folder_l2_list:
                    folder_l2_path = os.path.join(folder_l1_path, folder_l2)
                    roi_folder_list = os.listdir(folder_l2_path)
                    for roi_folder in roi_folder_list:
                        roi_infos = roi_folder.split('_')
                        roi_lat = float(roi_infos[2])
                        
                        if roi_lat < 0:
                        
                            roi_folder_path = os.path.join(folder_l2_path, roi_folder)
                            roi_file_list = os.listdir(roi_folder_path)
                            roi_misr_SR_band3_list = []
                            roi_ahi_SR_band3_list = []
                            roi_misr_SR_band4_list = []
                            roi_ahi_SR_band4_list = []
                            for roi_file in roi_file_list:
                                matchObj = re.search(r'(\d+)_band(\d+)_(\d+).npy', str(roi_file))
                                if matchObj:
                                    ahi_time_str = matchObj.group(1)
                                    band_str = matchObj.group(2)
                                    # camera_idx_str = matchObj.group(3)

                                    obs_month = ahi_time_str[4:6]
                                    obs_month_idx = int(obs_month) - 1

                                    if obs_month_idx == month_idx:
                                        SR_npy_path = os.path.join(roi_folder_path, roi_file)
                                        ROI_SR_pair = numpy.load(SR_npy_path, allow_pickle=True)[0]
                                        misr_sr = ROI_SR_pair['misr_v3']
                                        ahi_sr = ROI_SR_pair['ahi_sr2misr']
                                        x_3Darray_np_1d = misr_sr.flatten()
                                        x_3Darray_np_1d = x_3Darray_np_1d[~numpy.isnan(x_3Darray_np_1d)]
                                        y_3Darray_np_1d = ahi_sr.flatten()
                                        y_3Darray_np_1d = y_3Darray_np_1d[~numpy.isnan(y_3Darray_np_1d)]
                                        if band_str == '3':
                                            roi_misr_SR_band3_list.extend(x_3Darray_np_1d)
                                            roi_ahi_SR_band3_list.extend(y_3Darray_np_1d)
                                        if band_str == '4':
                                            roi_misr_SR_band4_list.extend(x_3Darray_np_1d)
                                            roi_ahi_SR_band4_list.extend(y_3Darray_np_1d)
                                    # keep pixel count same
                                    if len(roi_misr_SR_band3_list) == len(roi_misr_SR_band4_list):
                                        misr_SR_band3_item_list.extend(roi_misr_SR_band3_list)
                                        ahi_SR_band3_item_list.extend(roi_ahi_SR_band3_list)
                                        misr_SR_band4_item_list.extend(roi_misr_SR_band4_list)
                                        ahi_SR_band4_item_list.extend(roi_ahi_SR_band4_list)

                print('Month NO.:', month, 'Count:', len(misr_SR_band3_item_list))
                count_record.append(len(misr_SR_band3_item_list))
            for count_month in count_record:
                print(round(count_month/sum(count_record)*100, 2),end='% ')
            print('')

RAA
Month NO.: Jan Count: 2061
Month NO.: Feb Count: 243
Month NO.: Mar Count: 0
Month NO.: Apr Count: 891
Month NO.: May Count: 3537
Month NO.: Jun Count: 1809
Month NO.: Jul Count: 2979
Month NO.: Aug Count: 2286
Month NO.: Sep Count: 27
Month NO.: Oct Count: 18
Month NO.: Nov Count: 567
Month NO.: Dec Count: 4383
10.96% 1.29% 0.0% 4.74% 18.81% 9.62% 15.84% 12.16% 0.14% 0.1% 3.02% 23.31% 
Ray
Month NO.: Jan Count: 47790
Month NO.: Feb Count: 41769
Month NO.: Mar Count: 70641
Month NO.: Apr Count: 65943
Month NO.: May Count: 80550
Month NO.: Jun Count: 56034
Month NO.: Jul Count: 65025
Month NO.: Aug Count: 72585
Month NO.: Sep Count: 79434
Month NO.: Oct Count: 61920
Month NO.: Nov Count: 63936
Month NO.: Dec Count: 64026
6.21% 5.43% 9.18% 8.57% 10.47% 7.28% 8.45% 9.43% 10.32% 8.05% 8.31% 8.32% 


## 原始匹配到的

In [12]:
    folder_type_list = ['RAA', 'Ray']
    folder_l1_list = ['26']
    folder_l2_list = ['0']

    for folder_type in folder_type_list:
        print(folder_type)
        month_record = numpy.zeros((12,))
        folder_type_path = os.path.join(WORK_SPACE, folder_type)
        for folder_l1 in folder_l1_list:
            folder_l1_path = os.path.join(folder_type_path, folder_l1)
            for folder_l2 in folder_l2_list:
                folder_l2_path = os.path.join(folder_l1_path, folder_l2)
                roi_folder_list = os.listdir(folder_l2_path)
                for roi_folder in roi_folder_list:
                    roi_folder_path = os.path.join(folder_l2_path, roi_folder)
                    roi_infos = roi_folder.split('_')
                    roi_lat = float(roi_infos[2])
                    if roi_lat < 0:
                        matched_record_npy = os.path.join(roi_folder_path, roi_folder+'_matched_record.npy')
                        matched_record = numpy.load(matched_record_npy, allow_pickle=True)[0]
                        matched_infos = matched_record['roi_misr_infos']
                        for matched_info in matched_infos:
                            obs_time = matched_info['matched_info'][4]
                            month_idx = int(obs_time[4:6])-1
                            month_record[month_idx] = month_record[month_idx] + 1
        print(MONTH_LABEL)
        print(month_record)
        for count_month in month_record:
            print(round(count_month/sum(month_record)*100, 2),end='% ')
        print('')

RAA
['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
[241.  50.   0.  70. 125. 109. 110.  95.   5.   4. 110. 388.]
18.44% 3.83% 0.0% 5.36% 9.56% 8.34% 8.42% 7.27% 0.38% 0.31% 8.42% 29.69% 
Ray
['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
[494. 437. 500. 491. 508. 484. 499. 499. 498. 520. 491. 496.]
8.35% 7.39% 8.45% 8.3% 8.59% 8.18% 8.43% 8.43% 8.42% 8.79% 8.3% 8.38% 
